In [1]:
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
from itertools import product
import lightgbm as lgb
from sklearn.metrics import f1_score, roc_auc_score
import gc
import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', 200)

In [2]:
# count编码
def count_coding(df, fea_col):
    for f in fea_col:
        df[f + '_count'] = df[f].map(df[f].value_counts())
    return df

# 交叉特征统计
def cross_cat_num(df, num_col, cat_col):
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            feat = g[f2].agg({
                '{}_{}_max'.format(f1, f2): 'max',
                '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median'
            })
            df = df.merge(feat, on=f1, how='left')
    return df

In [3]:
class MeanEncoder:
    def __init__(self, categorical_features, n_splits=10, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode
 
        :param n_splits: the number of splits used in mean encoding
 
        :param target_type: str, 'regression' or 'classification'
 
        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """
 
        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}
 
        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None
 
        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))
 
    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()
 
        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()
 
        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg(['mean', 'size'])
        col_avg_y['size'] = prior_weight_func(col_avg_y['size'])
        col_avg_y[nf_name] = col_avg_y['size'] * prior + (1 - col_avg_y['size']) * col_avg_y['mean']
        col_avg_y.drop(['size', 'mean'], axis=1, inplace=True)
 
        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values
 
        return nf_train, nf_test, prior, col_avg_y
 
    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)
 
        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new
 
    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
 
        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
 
        return X_new

### 用户表

In [4]:
train_user = pd.read_hdf('../input/train_user.h5')
test_user = pd.read_hdf('../input/test_user.h5')

In [5]:
train_user.head()

phone_no_m  \
0  672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a3a835f6cccb78b8ed7554e9ab0fbcd33d19eb6063ce00542dd223cc5cc83c68f07bcf933547b6776b1   
1  5e1272273e041e82cb275ae877710be98cdaf5b0a8f34de8d361f71d5268fa0851edffd3950e170df1e3846fcf90cc7cc8299be9139a2ac4b5c5e5121d832674   
2  eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c682d6add360a4332a8ad294d8470d64a73c6e53e8413f0ad93b9ea65afab717e58d312554f33553ee7   
3  0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f3627cd4520064b93ec66a72d085a67f0f40eebb9f901072965073e398f04c4ae500a1db4dfb13a972c   
4  28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d7706b20520aa11fff0f4ce06203f66758ec0f0e70780e4e258aae6aa1f23439e115528e08eda71ed20   

  city_name county_name  idcard_cnt  label  arpu_202004  
0        绵阳       江油分公司           1      0    45.000000  
1        德阳       旌阳分公司           1      0    60.000000  
2        成都       金堂分公司           2      0    63.000000  
3        成都       高新分公司           2      0    23.203125  
4        德阳       旌阳分公司           1      0    50.000000

In [6]:
test_user.head()

phone_no_m  \
0  22d522340df77e2252c1a4d92b4bcb00d515e36f3ec6bf94e017b4ffd67e26520af18637ad91bde69abd831cc36cdc0c5af5e57bc280f95f79efef7bfa9c6926   
1  5220d4b8429bdba3971a7b46a088c6b8fa6710f40607598ac1219daab6071510b3f66f78637420574dbb2d5488d44de9d4360a644bbe4ec45aaa4b7513bdc886   
2  7d19dd2b50ced56f03d23bf928cf34dc570a48525571a868ffdcdc53a865b7a41c386a6f1d6567dd4a710530b933308c77d1ef89f77b580d1983dadd3de162ea   
3  134a4a591185c9d3788021896dcfc235e9e0a6a1e3f8a4fd3a9f9d9bf4a033e791b7199db980b0678020bdf689d719306a5c694ffa47b4ba2bec43ceda68dfc1   
4  2356dcd6759d50455ddaeed03c838843558e9182d5962f8a4c81a9a178063c49ab988750f795bb6c5462aee5dace2ad9fd18dbb1a6619dbb1e771e9b0f192da9   

  city_name county_name  idcard_cnt  arpu_202004  
0        成都       金牛分公司           1         9.00  
1      天府新区      天府直属部门           1          NaN  
2        成都       锦江分公司           1         9.00  
3        绵阳       江油分公司           1       138.75  
4        成都       青羊分公司           2        39.00

In [7]:
train_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6106 entries, 0 to 6105
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   phone_no_m   6106 non-null   object 
 1   city_name    6106 non-null   object 
 2   county_name  6106 non-null   object 
 3   idcard_cnt   6106 non-null   int8   
 4   label        6106 non-null   int8   
 5   arpu_202004  5369 non-null   float16
dtypes: float16(1), int8(2), object(3)
memory usage: 214.7+ KB


In [8]:
test_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2045 entries, 0 to 2044
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   phone_no_m   2045 non-null   object 
 1   city_name    2045 non-null   object 
 2   county_name  2045 non-null   object 
 3   idcard_cnt   2045 non-null   int8   
 4   arpu_202004  1869 non-null   float16
dtypes: float16(1), int8(1), object(3)
memory usage: 69.9+ KB


In [9]:
df_user = pd.concat([train_user, test_user])

In [10]:
# 号码量*消费值
df_user['idcard_cnt*arpu_202004'] = df_user['idcard_cnt'] * df_user['arpu_202004']

In [11]:
# count编码
count_list = ['city_name', 'county_name', 'idcard_cnt']

df_user = count_coding(df_user, count_list)

In [12]:
lbl = LabelEncoder()

for f in tqdm(['city_name', 'county_name']):
    df_user[f] = df_user[f].fillna('NA')
    df_user[f] = lbl.fit_transform(df_user[f].astype(str))

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 250.68it/s]


In [13]:
cross_cat = ['city_name', 'county_name']
cross_num = ['idcard_cnt']
df_user = cross_cat_num(df_user, cross_num, cross_cat)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 111.41it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 80.21it/s]


### 电话表

In [14]:
train_voc = pd.read_hdf('../input/train_voc.h5')
test_voc = pd.read_hdf('../input/test_voc.h5')

In [15]:
train_voc.head()

phone_no_m  \
9    b3994b4c46e39954cfe0cb8ebd2a79703a2ace6612fa18241509e8b2dfe9d75f503823b6df2d10180a9033ccd22390aaeda397037ca4cd26604d6db6e3470874   
10   b3994b4c46e39954cfe0cb8ebd2a79703a2ace6612fa18241509e8b2dfe9d75f503823b6df2d10180a9033ccd22390aaeda397037ca4cd26604d6db6e3470874   
219  c5502a6d3e4cabaf8c0e298c4aac693b6e80835d50d4e4f393d8426bd49c0d4d00c3959d2a2c8341410e57af28bee6dc5c80dbad7cf147018ec0ece7bfa0cf5f   
222  c5502a6d3e4cabaf8c0e298c4aac693b6e80835d50d4e4f393d8426bd49c0d4d00c3959d2a2c8341410e57af28bee6dc5c80dbad7cf147018ec0ece7bfa0cf5f   
276  2bdbc0a45d6228970b353d691e2ec229f0cf60790a1a39d5e01c0175b5e20cd2ca7d39aff8506741884fd7a4cd4fd9fdec5112ddb764d81c2983752c989f1078   

                                                                                                                        opposite_no_m  \
9    1b01124a6ad0443ad868757c6594a605a0c5179c2b5b57071d5cec8cd8c4c89d7598587f953f08887965b42d2e465d08388c5465fe43b67f1e9ca5509c5529f2   
10   1b01124a6ad0443ad868757c6594a605a0c5179c2b5b57071d5cec8cd8c4c89d7598587f953f08887965b42d2e465d08388c5465fe43b67f1e9ca5509c5529f2   
219  f1b73c46a691c068bfd38253e20e882d4993c8d3e58016aeed804a8cdbf54542112e529ebf7fdd0e62cf3556571fb17b865d8c3d57c42be851d75c90f4df4603   
222  5bc6d97a3017578edfa7979fa43653cd74bad827f18337c8c84728f79778939b6f939ea3aa2661907f2e79bcf70feb93adb79c3f7d4a934368c459d9b6aeb54e   
276  c42eeddeff98feb87860441ea6548f8bda2a080c935e02c7a6ae30223cf37dfc2d16760224c79b93ea44a6d3fd77139e03a20f47608b9455372a96cb026f239e   

     calltype_id       start_datetime  call_dur city_name county_name  \
9              1  2020-03-22 21:38:40       175       nan         nan   
10             1  2020-03-23 18:05:41        33       nan         nan   
219            1  2020-03-01 11:24:37        39       nan         nan   
222            1  2020-03-01 11:23:44        32       nan         nan   
276            2  2020-03-03 15:34:21         3       nan         nan   

                                                                                                                               imei_m  
9    90ba61111dad175330029e10da8264a86dfdb3a281ccb8556eca87a6f1af32e1d5ff4db4a7d4b11f760e4866de974933f247c4c389073f28634229889bfe18bf  
10   90ba61111dad175330029e10da8264a86dfdb3a281ccb8556eca87a6f1af32e1d5ff4db4a7d4b11f760e4866de974933f247c4c389073f28634229889bfe18bf  
219  e1e1149b2c06e9972293c457293abb32ad05b0c82b3b64c3786896b0414c897b60a539329ce3b7df83ebee19ad20fee3971303edd776a593578f03d16ee8981a  
222  e1e1149b2c06e9972293c457293abb32ad05b0c82b3b64c3786896b0414c897b60a539329ce3b7df83ebee19ad20fee3971303edd776a593578f03d16ee8981a  
276  3e808a14fa2aa61524c70f262539d1c8532b8e3a8c7c87c863b3f8420750f1bf4a63aa900aef09620c1fae1a6a7fadf1f42e30477e24336fc5c2a8fae9b5de6e

In [16]:
test_voc.head()

phone_no_m  \
0  b3dce36871f3e88164b18d4953b114163f008cb51c28fe932ae3c734f3b1d1e2853d63fb3fb52a09fd9d0997b64fe5796507d3b50768fe0dce23819c8b24729c   
1  b3dce36871f3e88164b18d4953b114163f008cb51c28fe932ae3c734f3b1d1e2853d63fb3fb52a09fd9d0997b64fe5796507d3b50768fe0dce23819c8b24729c   
2  b3dce36871f3e88164b18d4953b114163f008cb51c28fe932ae3c734f3b1d1e2853d63fb3fb52a09fd9d0997b64fe5796507d3b50768fe0dce23819c8b24729c   
3  7ec68a368fbbec3279a6a34847f7959623dbff4638351aa0c4ef3e6c719a4cce873e332d11c2c8f860a67edcc4bc3706da02976e26c218474c67a3ed5cdffed5   
4  7ec68a368fbbec3279a6a34847f7959623dbff4638351aa0c4ef3e6c719a4cce873e332d11c2c8f860a67edcc4bc3706da02976e26c218474c67a3ed5cdffed5   

                                                                                                                      opposite_no_m  \
0  f87f526ee776ac8b6b28392620fbb6049af9eaadab7de2ee9d7c5a31d7404566b8a3c25f404f793e513448a8c5dad1940a597a488e6f165a1aad6b103cf40f59   
1  f87f526ee776ac8b6b28392620fbb6049af9eaadab7de2ee9d7c5a31d7404566b8a3c25f404f793e513448a8c5dad1940a597a488e6f165a1aad6b103cf40f59   
2  f87f526ee776ac8b6b28392620fbb6049af9eaadab7de2ee9d7c5a31d7404566b8a3c25f404f793e513448a8c5dad1940a597a488e6f165a1aad6b103cf40f59   
3  ed8280e5590e40e35a2af7d8708a0754540b0612600d5255b8b79e576d1543765e552e30cb50aae3f4d424e08e8104a848f021316949d72e20a9efe0c29eba7c   
4  ba792e76c3e7ee47d2403083f6def313372ae450ddf0b946d2f3d237e05cd99e75265be9d91ce58bc149919c97e1355f56082da790dccb9cffb0d7e11c4fa7ed   

   calltype_id       start_datetime  call_dur city_name county_name  \
0            1  2020-04-13 21:04:10      1909       nan         nan   
1            1  2020-04-13 18:32:50      2510       nan         nan   
2            1  2020-04-13 19:55:12        25       nan         nan   
3            1  2020-04-20 14:11:44       276       nan         nan   
4            2  2020-04-20 12:56:33       539       nan         nan   

                                                                                                                             imei_m  
0  685a0bfcd91b4ecec2ff35d656a35a4e4f9e1c7a92b15f5c3c0675fcf8b6c9adc35731ab15e91401f2367f1d9d2910f9e8829e2e5052c96bc800fcde6c505214  
1  685a0bfcd91b4ecec2ff35d656a35a4e4f9e1c7a92b15f5c3c0675fcf8b6c9adc35731ab15e91401f2367f1d9d2910f9e8829e2e5052c96bc800fcde6c505214  
2  685a0bfcd91b4ecec2ff35d656a35a4e4f9e1c7a92b15f5c3c0675fcf8b6c9adc35731ab15e91401f2367f1d9d2910f9e8829e2e5052c96bc800fcde6c505214  
3  a60e82a4383faa79ea972adc48686a729f689b7d1923bce82ca67c7deae306fec30004b30260b56e13d77f791271222594954cc4f88a65c3348b97a9eeb47ad2  
4  a60e82a4383faa79ea972adc48686a729f689b7d1923bce82ca67c7deae306fec30004b30260b56e13d77f791271222594954cc4f88a65c3348b97a9eeb47ad2

In [17]:
train_voc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709144 entries, 9 to 5015413
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   phone_no_m      709144 non-null  object
 1   opposite_no_m   709144 non-null  object
 2   calltype_id     709144 non-null  int8  
 3   start_datetime  709144 non-null  object
 4   call_dur        709144 non-null  int16 
 5   city_name       709144 non-null  object
 6   county_name     709144 non-null  object
 7   imei_m          709144 non-null  object
dtypes: int16(1), int8(1), object(6)
memory usage: 39.9+ MB


In [18]:
test_voc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276522 entries, 0 to 276521
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   phone_no_m      276522 non-null  object
 1   opposite_no_m   276522 non-null  object
 2   calltype_id     276522 non-null  int8  
 3   start_datetime  276522 non-null  object
 4   call_dur        276522 non-null  int16 
 5   city_name       276522 non-null  object
 6   county_name     276522 non-null  object
 7   imei_m          276522 non-null  object
dtypes: int16(1), int8(1), object(6)
memory usage: 15.6+ MB


In [19]:
train_voc.nunique()

phone_no_m          4823
opposite_no_m     281103
calltype_id            3
start_datetime    561615
call_dur            2653
city_name             22
county_name          193
imei_m              6025
dtype: int64

In [20]:
test_voc.nunique()

phone_no_m          1965
opposite_no_m     107683
calltype_id            3
start_datetime    253144
call_dur            2089
city_name             22
county_name          173
imei_m              2656
dtype: int64

In [21]:
df_voc = pd.concat([train_voc, test_voc])

In [22]:
df_voc['voc_day'] = df_voc['start_datetime'].astype('datetime64').dt.day
df_voc['voc_hour'] = df_voc['start_datetime'].astype('datetime64').dt.hour
df_voc['voc_dayofweek'] = df_voc['start_datetime'].astype('datetime64').dt.dayofweek

In [23]:
df_voc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985666 entries, 9 to 276521
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   phone_no_m      985666 non-null  object
 1   opposite_no_m   985666 non-null  object
 2   calltype_id     985666 non-null  int8  
 3   start_datetime  985666 non-null  object
 4   call_dur        985666 non-null  int16 
 5   city_name       985666 non-null  object
 6   county_name     985666 non-null  object
 7   imei_m          985666 non-null  object
 8   voc_day         985666 non-null  int64 
 9   voc_hour        985666 non-null  int64 
 10  voc_dayofweek   985666 non-null  int64 
dtypes: int16(1), int64(3), int8(1), object(6)
memory usage: 78.0+ MB


In [24]:
# 每天通话次数统计
df_voc['voc_day_count'] = df_voc.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].transform('count')
df_voc['voc_day_count_max'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('max')
df_voc['voc_day_count_min'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('min')
df_voc['voc_day_count_mean'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('mean')
df_voc['voc_day_count_std'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('std')




# 每天与不同的人通话次数统计
df_voc['voc_day_diff_count'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_day'])['phone_no_m'].transform('count')
df_voc['voc_day_diff_count_max'] = df_voc.groupby('phone_no_m')['voc_day_diff_count'].transform('max')
df_voc['voc_day_diff_count_min'] = df_voc.groupby('phone_no_m')['voc_day_diff_count'].transform('min')
df_voc['voc_day_diff_count_mean'] = df_voc.groupby('phone_no_m')['voc_day_diff_count'].transform('mean')
df_voc['voc_day_diff_count_std'] = df_voc.groupby('phone_no_m')['voc_day_diff_count'].transform('std')

del df_voc['voc_day_count'], df_voc['voc_day_diff_count']
gc.collect()


# 每个phone_no_m在几号的通话次数
voc_res = df_voc.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].count().unstack()

for i in df_voc['voc_day'].unique():
    df_voc['voc_day{}_count'.format(i)] = df_voc['phone_no_m'].map(voc_res[i])

In [25]:
# 每天通话时长统计
df_voc['call_dur_day_sum'] = df_voc.groupby(['phone_no_m', 'voc_day'])['call_dur'].transform('sum')
df_voc['call_dur_day_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_day_sum'].transform('max')
df_voc['call_dur_day_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_day_sum'].transform('min')
df_voc['call_dur_day_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_day_sum'].transform('mean')
df_voc['call_dur_day_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_day_sum'].transform('std')


# 每天与不同的人通话时长统计
df_voc['call_dur_diff_sum'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_day'])['call_dur'].transform('sum')
df_voc['call_dur_diff_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_diff_sum'].transform('max')
df_voc['call_dur_diff_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_diff_sum'].transform('min')
df_voc['call_dur_diff_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_diff_sum'].transform('mean')
df_voc['call_dur_diff_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_diff_sum'].transform('std')

del df_voc['call_dur_day_sum'], df_voc['call_dur_diff_sum']
gc.collect()


# 每个phone_no_m在几号的通话时长
voc_day_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_day'])['call_dur'].sum().unstack()

for i in df_voc['voc_day'].unique():
    df_voc['voc_day_call_dur{}_sum'.format(i)] = df_voc['phone_no_m'].map(voc_day_call_dur_res[i])

In [26]:
# 一小时内通话次数

df_voc['voc_hour_count'] = df_voc.groupby(['phone_no_m', 'voc_hour'])['phone_no_m'].transform('count')
df_voc['voc_hour_count_max'] = df_voc.groupby('phone_no_m')['voc_hour_count'].transform('max')
df_voc['voc_hour_count_min'] = df_voc.groupby('phone_no_m')['voc_hour_count'].transform('min')
df_voc['voc_hour_count_mean'] = df_voc.groupby('phone_no_m')['voc_hour_count'].transform('mean')
df_voc['voc_hour_count_std'] = df_voc.groupby('phone_no_m')['voc_hour_count'].transform('std')




# 一小时内与不同的人通话次数

df_voc['voc_hour_diff_count'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_hour'])['phone_no_m'].transform('count')
df_voc['voc_hour_diff_count_max'] = df_voc.groupby('phone_no_m')['voc_hour_diff_count'].transform('max')
df_voc['voc_hour_diff_count_min'] = df_voc.groupby('phone_no_m')['voc_hour_diff_count'].transform('min')
df_voc['voc_hour_diff_count_mean'] = df_voc.groupby('phone_no_m')['voc_hour_diff_count'].transform('mean')
df_voc['voc_hour_diff_count_std'] = df_voc.groupby('phone_no_m')['voc_hour_diff_count'].transform('std')

del df_voc['voc_hour_count'], df_voc['voc_hour_diff_count']
gc.collect()


# 每个phone_no_m在几号的通话时长
voc_hour_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_hour'])['call_dur'].sum().unstack()

for i in df_voc['voc_hour'].unique():
    df_voc['voc_hour_call_dur{}_sum'.format(i)] = df_voc['phone_no_m'].map(voc_hour_call_dur_res[i])

In [27]:
# 周几通话统计量

df_voc['voc_dayofweek_count'] = df_voc.groupby(['phone_no_m', 'voc_dayofweek'])['phone_no_m'].transform('count')
df_voc['voc_dayofweek_count_max'] = df_voc.groupby('phone_no_m')['voc_dayofweek'].transform('max')
df_voc['voc_dayofweek_count_min'] = df_voc.groupby('phone_no_m')['voc_dayofweek'].transform('min')
df_voc['voc_dayofweek_count_mean'] = df_voc.groupby('phone_no_m')['voc_dayofweek'].transform('mean')
df_voc['voc_dayofweek_count_std'] = df_voc.groupby('phone_no_m')['voc_dayofweek'].transform('std')

del df_voc['voc_dayofweek_count']
gc.collect()

20

In [28]:
# 该月总的通话次数
df_voc['voc_count'] = df_voc.groupby('phone_no_m')['phone_no_m'].transform('count')

# 相互打电话次数
df_voc['voc_count_mutual'] = df_voc.groupby(['phone_no_m', 'opposite_no_m'])['phone_no_m'].transform('count')

In [29]:
# del df_voc['voc_hour'], df_voc['voc_dayofweek']
# gc.collect()

### 短信表

In [30]:
train_sms = pd.read_hdf('../input/train_sms.h5')
test_sms = pd.read_hdf('../input/test_sms.h5')

In [31]:
train_sms = train_sms[train_sms['request_datetime'] >= '2020-03-01 00:00:00']

In [32]:
df_sms = pd.concat([train_sms, test_sms])

In [33]:
df_sms.rename(columns={'calltype_id': 'calltype_id_sms'}, inplace=True)

In [34]:
df_sms['sms_day'] = df_sms['request_datetime'].astype('datetime64').dt.day
df_sms['sms_hour'] = df_sms['request_datetime'].astype('datetime64').dt.hour
df_sms['sms_dayofweek'] = df_sms['request_datetime'].astype('datetime64').dt.dayofweek

In [35]:
# 一天内短信统计量

df_sms['sms_day_count'] = df_sms.groupby(['phone_no_m', 'sms_day'])['phone_no_m'].transform('count')
df_sms['sms_day_count_max'] = df_sms.groupby('phone_no_m')['sms_day_count'].transform('max')
df_sms['sms_day_count_min'] = df_sms.groupby('phone_no_m')['sms_day_count'].transform('min')
df_sms['sms_day_count_mean'] = df_sms.groupby('phone_no_m')['sms_day_count'].transform('mean')
df_sms['sms_day_count_std'] = df_sms.groupby('phone_no_m')['sms_day_count'].transform('std')

del df_sms['sms_day_count']


# 一天内与不同的人短信统计量
df_sms['sms_day_diff_count'] = df_sms.groupby(['phone_no_m', 'opposite_no_m', 'sms_day'])['phone_no_m'].transform('count')
df_sms['sms_day_diff_count_max'] = df_sms.groupby('phone_no_m')['sms_day_diff_count'].transform('max')
df_sms['sms_day_diff_count_min'] = df_sms.groupby('phone_no_m')['sms_day_diff_count'].transform('min')
df_sms['sms_day_diff_count_mean'] = df_sms.groupby('phone_no_m')['sms_day_diff_count'].transform('mean')
df_sms['sms_day_diff_count_std'] = df_sms.groupby('phone_no_m')['sms_day_diff_count'].transform('std')

del df_sms['sms_day_diff_count']
gc.collect()

11

In [36]:
sms_res = df_sms.groupby(['phone_no_m', 'sms_day'])['phone_no_m'].count().unstack()

for i in df_sms['sms_day'].unique():
    df_sms['sms_day{}_count'.format(i)] = df_sms['phone_no_m'].map(sms_res[i])

In [37]:
# 一小时内短信统计量

df_sms['sms_hour_count'] = df_sms.groupby(['phone_no_m', 'sms_hour'])['phone_no_m'].transform('count')
df_sms['sms_hour_count_max'] = df_sms.groupby('phone_no_m')['sms_hour_count'].transform('max')
df_sms['sms_hour_count_min'] = df_sms.groupby('phone_no_m')['sms_hour_count'].transform('min')
df_sms['sms_hour_count_mean'] = df_sms.groupby('phone_no_m')['sms_hour_count'].transform('mean')
df_sms['sms_hour_count_std'] = df_sms.groupby('phone_no_m')['sms_hour_count'].transform('std')

del df_sms['sms_hour_count']


# 一小时内与不同的人短信统计量
df_sms['sms_hour_diff_count'] = df_sms.groupby(['phone_no_m', 'opposite_no_m', 'sms_hour'])['phone_no_m'].transform('count')
df_sms['sms_hour_diff_count_max'] = df_sms.groupby('phone_no_m')['sms_hour_diff_count'].transform('max')
df_sms['sms_hour_diff_count_min'] = df_sms.groupby('phone_no_m')['sms_hour_diff_count'].transform('min')
df_sms['sms_hour_diff_count_mean'] = df_sms.groupby('phone_no_m')['sms_hour_diff_count'].transform('mean')
df_sms['sms_hour_diff_count_std'] = df_sms.groupby('phone_no_m')['sms_hour_diff_count'].transform('std')

del df_sms['sms_hour_diff_count']
gc.collect()

0

In [38]:
# 周几短信统计量

df_sms['sms_dayofweek_count'] = df_sms.groupby(['phone_no_m', 'sms_dayofweek'])['phone_no_m'].transform('count')
df_sms['sms_dayofweek_count_max'] = df_sms.groupby('phone_no_m')['sms_dayofweek_count'].transform('max')
df_sms['sms_dayofweek_count_min'] = df_sms.groupby('phone_no_m')['sms_dayofweek_count'].transform('min')
df_sms['sms_dayofweek_count_mean'] = df_sms.groupby('phone_no_m')['sms_dayofweek_count'].transform('mean')
df_sms['sms_dayofweek_count_std'] = df_sms.groupby('phone_no_m')['sms_dayofweek_count'].transform('std')

del df_sms['sms_dayofweek_count']
gc.collect()

20

In [39]:
# 该月总的短信次数
df_sms['sms_count'] = df_sms.groupby('phone_no_m')['phone_no_m'].transform('count')

# 相互发送短信次数
df_sms['sms_count_mutual'] = df_sms.groupby(['phone_no_m', 'opposite_no_m'])['phone_no_m'].transform('count')

In [40]:
# del df_sms['sms_day'], df_sms['sms_hour'], df_sms['sms_dayofweek']
# gc.collect()

### 应用表

In [41]:
train_app = pd.read_hdf('../input/train_app.h5')
test_app = pd.read_hdf('../input/test_app.h5')

In [42]:
train_app = train_app[train_app['month_id'] == '2020-03']

In [43]:
df_app = pd.concat([train_app, test_app])

In [44]:
# 用户流量统计

df_app['total_flow'] = df_app.groupby('phone_no_m')['flow'].transform('sum')
df_app['flow_max'] = df_app.groupby('phone_no_m')['flow'].transform('max')
df_app['flow_min'] = df_app.groupby('phone_no_m')['flow'].transform('min')
df_app['flow_mean'] = df_app.groupby('phone_no_m')['flow'].transform('mean')
df_app['flow_std'] = df_app.groupby('phone_no_m')['flow'].transform('std')

# 应用数
df_app['app_count'] = df_app.groupby('phone_no_m')['phone_no_m'].transform('count')

In [45]:
lbl = LabelEncoder()

for f in tqdm(['busi_name']):
    df_app[f] = df_app[f].fillna('NA')
    df_app[f] = lbl.fit_transform(df_app[f].astype(str))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.93it/s]


In [46]:
# 用户在每个app上的流量统计

df_app['busi_name_total_flow'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('sum')
df_app['busi_name_flow_max'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('max')
df_app['busi_name_flow_min'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('min')
df_app['busi_name_flow_mean'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('mean')
df_app['busi_name_flow_std'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('std')

### 合并数据

In [47]:
df_user.shape, df_voc.shape, df_sms.shape, df_app.shape

((8151, 16), (985666, 127), (1319506, 60), (512100, 15))

In [48]:
print(df_user.columns.tolist())

['phone_no_m', 'city_name', 'county_name', 'idcard_cnt', 'label', 'arpu_202004', 'idcard_cnt*arpu_202004', 'city_name_count', 'county_name_count', 'idcard_cnt_count', 'city_name_idcard_cnt_max', 'city_name_idcard_cnt_min', 'city_name_idcard_cnt_median', 'county_name_idcard_cnt_max', 'county_name_idcard_cnt_min', 'county_name_idcard_cnt_median']


In [49]:
print(df_voc.columns.tolist())

['phone_no_m', 'opposite_no_m', 'calltype_id', 'start_datetime', 'call_dur', 'city_name', 'county_name', 'imei_m', 'voc_day', 'voc_hour', 'voc_dayofweek', 'voc_day_count_max', 'voc_day_count_min', 'voc_day_count_mean', 'voc_day_count_std', 'voc_day_diff_count_max', 'voc_day_diff_count_min', 'voc_day_diff_count_mean', 'voc_day_diff_count_std', 'voc_day22_count', 'voc_day23_count', 'voc_day1_count', 'voc_day3_count', 'voc_day9_count', 'voc_day6_count', 'voc_day10_count', 'voc_day26_count', 'voc_day27_count', 'voc_day13_count', 'voc_day28_count', 'voc_day30_count', 'voc_day20_count', 'voc_day16_count', 'voc_day19_count', 'voc_day21_count', 'voc_day29_count', 'voc_day17_count', 'voc_day7_count', 'voc_day14_count', 'voc_day15_count', 'voc_day5_count', 'voc_day2_count', 'voc_day12_count', 'voc_day8_count', 'voc_day18_count', 'voc_day31_count', 'voc_day4_count', 'voc_day24_count', 'voc_day25_count', 'voc_day11_count', 'call_dur_day_sum_max', 'call_dur_day_sum_min', 'call_dur_day_sum_mean', 'c

In [50]:
voc_cols = [i for i in df_voc.columns if i not in ['opposite_no_m', 'start_datetime', 'city_name', 'county_name', 'imei_m']]

In [51]:
df_voc = df_voc[voc_cols].drop_duplicates(subset=['phone_no_m'])

In [52]:
print(df_sms.columns.tolist())

['phone_no_m', 'opposite_no_m', 'calltype_id_sms', 'request_datetime', 'sms_day', 'sms_hour', 'sms_dayofweek', 'sms_day_count_max', 'sms_day_count_min', 'sms_day_count_mean', 'sms_day_count_std', 'sms_day_diff_count_max', 'sms_day_diff_count_min', 'sms_day_diff_count_mean', 'sms_day_diff_count_std', 'sms_day1_count', 'sms_day2_count', 'sms_day3_count', 'sms_day4_count', 'sms_day5_count', 'sms_day6_count', 'sms_day7_count', 'sms_day8_count', 'sms_day9_count', 'sms_day10_count', 'sms_day11_count', 'sms_day12_count', 'sms_day13_count', 'sms_day14_count', 'sms_day15_count', 'sms_day16_count', 'sms_day17_count', 'sms_day18_count', 'sms_day19_count', 'sms_day20_count', 'sms_day21_count', 'sms_day22_count', 'sms_day23_count', 'sms_day24_count', 'sms_day25_count', 'sms_day26_count', 'sms_day27_count', 'sms_day28_count', 'sms_day29_count', 'sms_day30_count', 'sms_day31_count', 'sms_hour_count_max', 'sms_hour_count_min', 'sms_hour_count_mean', 'sms_hour_count_std', 'sms_hour_diff_count_max', 'sm

In [53]:
sms_cols = [i for i in df_sms.columns if i not in ['opposite_no_m', 'request_datetime']]

In [54]:
df_sms = df_sms[sms_cols].drop_duplicates(subset=['phone_no_m'])

In [55]:
print(df_app.columns.tolist())

['phone_no_m', 'busi_name', 'flow', 'month_id', 'total_flow', 'flow_max', 'flow_min', 'flow_mean', 'flow_std', 'app_count', 'busi_name_total_flow', 'busi_name_flow_max', 'busi_name_flow_min', 'busi_name_flow_mean', 'busi_name_flow_std']


In [56]:
app_cols = [i for i in df_app.columns if i not in ['month_id']]

In [57]:
df_app = df_app[app_cols].drop_duplicates(subset=['phone_no_m'])

In [58]:
df = pd.merge(df_user, df_voc, how='left', on='phone_no_m')
df = pd.merge(df, df_sms, how='left', on='phone_no_m')
df = pd.merge(df, df_app, how='left', on='phone_no_m')

In [59]:
del df_user, df_voc, df_sms, df_app
gc.collect()

20

In [60]:
print(df.columns.tolist())

['phone_no_m', 'city_name', 'county_name', 'idcard_cnt', 'label', 'arpu_202004', 'idcard_cnt*arpu_202004', 'city_name_count', 'county_name_count', 'idcard_cnt_count', 'city_name_idcard_cnt_max', 'city_name_idcard_cnt_min', 'city_name_idcard_cnt_median', 'county_name_idcard_cnt_max', 'county_name_idcard_cnt_min', 'county_name_idcard_cnt_median', 'calltype_id', 'call_dur', 'voc_day', 'voc_hour', 'voc_dayofweek', 'voc_day_count_max', 'voc_day_count_min', 'voc_day_count_mean', 'voc_day_count_std', 'voc_day_diff_count_max', 'voc_day_diff_count_min', 'voc_day_diff_count_mean', 'voc_day_diff_count_std', 'voc_day22_count', 'voc_day23_count', 'voc_day1_count', 'voc_day3_count', 'voc_day9_count', 'voc_day6_count', 'voc_day10_count', 'voc_day26_count', 'voc_day27_count', 'voc_day13_count', 'voc_day28_count', 'voc_day30_count', 'voc_day20_count', 'voc_day16_count', 'voc_day19_count', 'voc_day21_count', 'voc_day29_count', 'voc_day17_count', 'voc_day7_count', 'voc_day14_count', 'voc_day15_count', 'v

In [61]:
df.columns

Index(['phone_no_m', 'city_name', 'county_name', 'idcard_cnt', 'label',
       'arpu_202004', 'idcard_cnt*arpu_202004', 'city_name_count',
       'county_name_count', 'idcard_cnt_count',
       ...
       'flow_max', 'flow_min', 'flow_mean', 'flow_std', 'app_count',
       'busi_name_total_flow', 'busi_name_flow_max', 'busi_name_flow_min',
       'busi_name_flow_mean', 'busi_name_flow_std'],
      dtype='object', length=207)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8151 entries, 0 to 8150
Columns: 207 entries, phone_no_m to busi_name_flow_std
dtypes: float16(2), float32(11), float64(182), int32(2), int64(3), int8(6), object(1)
memory usage: 12.1+ MB


In [63]:
df_train = df[df.label.notna()]
df_test = df[df.label.isna()]

df_train.shape, df_test.shape

((6106, 207), (2045, 207))

In [64]:
df_train_train = df_train[df_train['voc_day'] <= 23]
df_train_valid = df_train[df_train['voc_day'] > 23]

X_train = df_train_train.drop('label', axis=1)
y_train = df_train_train['label']
X_valid = df_train_valid.drop('label', axis=1)
y_valid = df_train_valid['label']

In [65]:
train_cols = [i for i in X_train if i not in ['phone_no_m', 'label']]

In [66]:
params = {'objective': 'binary',
          'boosting': 'gbdt',
          'metric': 'auc',
          'learning_rate': 0.1,
          'num_leaves': 31,
          'lambda_l1': 0,
          'lambda_l2': 1,
          'min_data_in_leaf': 20,
          'is_unbalance': True,
          'max_depth': -1,
          'seed': 2020}

In [67]:
use_cols = []
useless_cols = []

for i in train_cols:
    print(i)
    
    lgb_train = lgb.Dataset(X_train[[i]].values, y_train) 
    lgb_eval= lgb.Dataset(X_valid[[i]].values, y_valid, reference=lgb_train)
    lgb_test = lgb.train(params,
                         lgb_train,
                         num_boost_round=1000,
                         valid_sets=[lgb_eval, lgb_train],
                         early_stopping_rounds=50,
                         verbose_eval=1)
    
    print('*' * 5)
    print(lgb_test.best_score['valid_0']['auc'])
    if lgb_test.best_score['valid_0']['auc'] > 0.52:
        use_cols.append(i)
    else:
        useless_cols.append(i)
    print('*' * 20)
    print('\n')

city_name
[1]	training's auc: 0.64515	valid_0's auc: 0.601982
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.646017	valid_0's auc: 0.599085
[3]	training's auc: 0.646017	valid_0's auc: 0.599085
[4]	training's auc: 0.646056	valid_0's auc: 0.599196
[5]	training's auc: 0.64628	valid_0's auc: 0.596126
[6]	training's auc: 0.64628	valid_0's auc: 0.596126
[7]	training's auc: 0.646313	valid_0's auc: 0.595573
[8]	training's auc: 0.646313	valid_0's auc: 0.595573
[9]	training's auc: 0.646313	valid_0's auc: 0.595573
[10]	training's auc: 0.646313	valid_0's auc: 0.595573
[11]	training's auc: 0.646313	valid_0's auc: 0.595573
[12]	training's auc: 0.646313	valid_0's auc: 0.595573
[13]	training's auc: 0.646313	valid_0's auc: 0.595573
[14]	training's auc: 0.646321	valid_0's auc: 0.597995
[15]	training's auc: 0.646321	valid_0's auc: 0.597995
[16]	training's auc: 0.646321	valid_0's auc: 0.597995
[17]	training's auc: 0.646321	valid_0's auc: 0.597995
[18]	training's auc: 0.

[3]	training's auc: 0.706151	valid_0's auc: 0.625016
[4]	training's auc: 0.70824	valid_0's auc: 0.623991
[5]	training's auc: 0.712133	valid_0's auc: 0.630661
[6]	training's auc: 0.712468	valid_0's auc: 0.629535
[7]	training's auc: 0.716159	valid_0's auc: 0.627363
[8]	training's auc: 0.716754	valid_0's auc: 0.627256
[9]	training's auc: 0.724177	valid_0's auc: 0.624144
[10]	training's auc: 0.7253	valid_0's auc: 0.623848
[11]	training's auc: 0.72616	valid_0's auc: 0.622018
[12]	training's auc: 0.726034	valid_0's auc: 0.620124
[13]	training's auc: 0.727831	valid_0's auc: 0.622874
[14]	training's auc: 0.729042	valid_0's auc: 0.623356
[15]	training's auc: 0.729523	valid_0's auc: 0.622044
[16]	training's auc: 0.730267	valid_0's auc: 0.621787
[17]	training's auc: 0.730734	valid_0's auc: 0.620866
[18]	training's auc: 0.732068	valid_0's auc: 0.622051
[19]	training's auc: 0.732214	valid_0's auc: 0.620016
[20]	training's auc: 0.734007	valid_0's auc: 0.619772
[21]	training's auc: 0.735764	valid_0's

[49]	training's auc: 0.644041	valid_0's auc: 0.602956
[50]	training's auc: 0.644041	valid_0's auc: 0.602956
[51]	training's auc: 0.644041	valid_0's auc: 0.602956
Early stopping, best iteration is:
[1]	training's auc: 0.642902	valid_0's auc: 0.603682
*****
0.603681640625
********************


county_name_count
[1]	training's auc: 0.726267	valid_0's auc: 0.621556
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.727248	valid_0's auc: 0.620527
[3]	training's auc: 0.727263	valid_0's auc: 0.617904
[4]	training's auc: 0.728799	valid_0's auc: 0.619993
[5]	training's auc: 0.729559	valid_0's auc: 0.618747
[6]	training's auc: 0.729467	valid_0's auc: 0.618838
[7]	training's auc: 0.731263	valid_0's auc: 0.618047
[8]	training's auc: 0.731577	valid_0's auc: 0.61804
[9]	training's auc: 0.7317	valid_0's auc: 0.618604
[10]	training's auc: 0.732189	valid_0's auc: 0.617988
[11]	training's auc: 0.733018	valid_0's auc: 0.615641
[12]	training's auc: 0.733152	valid_0's auc: 

[4]	training's auc: 0.554873	valid_0's auc: 0.54724
[5]	training's auc: 0.554873	valid_0's auc: 0.54724
[6]	training's auc: 0.554873	valid_0's auc: 0.54724
[7]	training's auc: 0.554873	valid_0's auc: 0.54724
[8]	training's auc: 0.554873	valid_0's auc: 0.54724
[9]	training's auc: 0.554873	valid_0's auc: 0.54724
[10]	training's auc: 0.554873	valid_0's auc: 0.54724
[11]	training's auc: 0.554873	valid_0's auc: 0.54724
[12]	training's auc: 0.554873	valid_0's auc: 0.54724
[13]	training's auc: 0.554873	valid_0's auc: 0.54724
[14]	training's auc: 0.554873	valid_0's auc: 0.54724
[15]	training's auc: 0.554873	valid_0's auc: 0.54724
[16]	training's auc: 0.554873	valid_0's auc: 0.54724
[17]	training's auc: 0.554873	valid_0's auc: 0.54724
[18]	training's auc: 0.554873	valid_0's auc: 0.54724
[19]	training's auc: 0.554873	valid_0's auc: 0.54724
[20]	training's auc: 0.554873	valid_0's auc: 0.54724
[21]	training's auc: 0.554873	valid_0's auc: 0.54724
[22]	training's auc: 0.554873	valid_0's auc: 0.54724

[1]	training's auc: 0.53452	valid_0's auc: 0.539844
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.53452	valid_0's auc: 0.539844
[3]	training's auc: 0.53452	valid_0's auc: 0.539844
[4]	training's auc: 0.53452	valid_0's auc: 0.539844
[5]	training's auc: 0.53452	valid_0's auc: 0.539844
[6]	training's auc: 0.53452	valid_0's auc: 0.539844
[7]	training's auc: 0.53452	valid_0's auc: 0.539844
[8]	training's auc: 0.53452	valid_0's auc: 0.539844
[9]	training's auc: 0.53452	valid_0's auc: 0.539844
[10]	training's auc: 0.53452	valid_0's auc: 0.539844
[11]	training's auc: 0.53452	valid_0's auc: 0.539844
[12]	training's auc: 0.53452	valid_0's auc: 0.539844
[13]	training's auc: 0.53452	valid_0's auc: 0.539844
[14]	training's auc: 0.53452	valid_0's auc: 0.539844
[15]	training's auc: 0.53452	valid_0's auc: 0.539844
[16]	training's auc: 0.53452	valid_0's auc: 0.539844
[17]	training's auc: 0.53452	valid_0's auc: 0.539844
[18]	training's auc: 0.53452	valid_0's auc: 0.5

...

*****
0.620390625
********************


flow_max
[1]	training's auc: 0.691104	valid_0's auc: 0.587904
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.702302	valid_0's auc: 0.587777
[3]	training's auc: 0.702837	valid_0's auc: 0.590667
[4]	training's auc: 0.703729	valid_0's auc: 0.585745
[5]	training's auc: 0.705175	valid_0's auc: 0.586071
[6]	training's auc: 0.706276	valid_0's auc: 0.588451
[7]	training's auc: 0.707293	valid_0's auc: 0.588763
[8]	training's auc: 0.708211	valid_0's auc: 0.586224
[9]	training's auc: 0.710833	valid_0's auc: 0.589333
[10]	training's auc: 0.711361	valid_0's auc: 0.588125
[11]	training's auc: 0.711342	valid_0's auc: 0.587897
[12]	training's auc: 0.711814	valid_0's auc: 0.588678
[13]	training's auc: 0.712206	valid_0's auc: 0.58776
[14]	training's auc: 0.712888	valid_0's auc: 0.590107
[15]	training's auc: 0.713445	valid_0's auc: 0.590309
[16]	training's auc: 0.714051	valid_0's auc: 0.589648
[17]	training's auc: 0.714292	valid

[15]	training's auc: 0.68413	valid_0's auc: 0.59877
[16]	training's auc: 0.685087	valid_0's auc: 0.59889
[17]	training's auc: 0.685741	valid_0's auc: 0.598896
[18]	training's auc: 0.686354	valid_0's auc: 0.596396
[19]	training's auc: 0.686833	valid_0's auc: 0.59766
[20]	training's auc: 0.687482	valid_0's auc: 0.599434
[21]	training's auc: 0.688173	valid_0's auc: 0.599808
[22]	training's auc: 0.688429	valid_0's auc: 0.60123
[23]	training's auc: 0.688883	valid_0's auc: 0.603418
[24]	training's auc: 0.689253	valid_0's auc: 0.60374
[25]	training's auc: 0.689688	valid_0's auc: 0.605016
[26]	training's auc: 0.689852	valid_0's auc: 0.60499
[27]	training's auc: 0.690746	valid_0's auc: 0.606312
[28]	training's auc: 0.691367	valid_0's auc: 0.608161
[29]	training's auc: 0.691599	valid_0's auc: 0.606888
[30]	training's auc: 0.691952	valid_0's auc: 0.60916
[31]	training's auc: 0.691911	valid_0's auc: 0.607441
[32]	training's auc: 0.692226	valid_0's auc: 0.608112
[33]	training's auc: 0.692943	valid_

[54]	training's auc: 0.756003	valid_0's auc: 0.606048
[55]	training's auc: 0.755991	valid_0's auc: 0.606042
[56]	training's auc: 0.756178	valid_0's auc: 0.606432
[57]	training's auc: 0.75624	valid_0's auc: 0.606406
[58]	training's auc: 0.756383	valid_0's auc: 0.605833
[59]	training's auc: 0.756482	valid_0's auc: 0.606452
[60]	training's auc: 0.756616	valid_0's auc: 0.606374
[61]	training's auc: 0.756854	valid_0's auc: 0.604395
[62]	training's auc: 0.756954	valid_0's auc: 0.603568
[63]	training's auc: 0.756973	valid_0's auc: 0.604499
[64]	training's auc: 0.757072	valid_0's auc: 0.602949
Early stopping, best iteration is:
[14]	training's auc: 0.744709	valid_0's auc: 0.617425
*****
0.6174251302083333
********************


busi_name_total_flow
[1]	training's auc: 0.692793	valid_0's auc: 0.580651
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.693934	valid_0's auc: 0.582035
[3]	training's auc: 0.695328	valid_0's auc: 0.596976
[4]	training's auc: 0.696001	

[77]	training's auc: 0.73413	valid_0's auc: 0.638086
[78]	training's auc: 0.734266	valid_0's auc: 0.63791
[79]	training's auc: 0.734404	valid_0's auc: 0.638174
[80]	training's auc: 0.734462	valid_0's auc: 0.637562
[81]	training's auc: 0.734954	valid_0's auc: 0.636846
[82]	training's auc: 0.735068	valid_0's auc: 0.63665
[83]	training's auc: 0.735209	valid_0's auc: 0.636396
[84]	training's auc: 0.73533	valid_0's auc: 0.635882
[85]	training's auc: 0.735437	valid_0's auc: 0.636012
[86]	training's auc: 0.735423	valid_0's auc: 0.635654
[87]	training's auc: 0.735428	valid_0's auc: 0.635186
[88]	training's auc: 0.735544	valid_0's auc: 0.634958
[89]	training's auc: 0.735468	valid_0's auc: 0.634684
[90]	training's auc: 0.735698	valid_0's auc: 0.635029
[91]	training's auc: 0.735783	valid_0's auc: 0.63542
[92]	training's auc: 0.735539	valid_0's auc: 0.636696
[93]	training's auc: 0.735643	valid_0's auc: 0.636305
[94]	training's auc: 0.736055	valid_0's auc: 0.63443
[95]	training's auc: 0.736133	vali

[31]	training's auc: 0.691589	valid_0's auc: 0.583545
[32]	training's auc: 0.691835	valid_0's auc: 0.582891
[33]	training's auc: 0.692187	valid_0's auc: 0.582884
[34]	training's auc: 0.692371	valid_0's auc: 0.582786
[35]	training's auc: 0.694141	valid_0's auc: 0.577712
[36]	training's auc: 0.69418	valid_0's auc: 0.578128
[37]	training's auc: 0.692932	valid_0's auc: 0.58224
[38]	training's auc: 0.693304	valid_0's auc: 0.582747
[39]	training's auc: 0.693459	valid_0's auc: 0.581895
[40]	training's auc: 0.694949	valid_0's auc: 0.579141
[41]	training's auc: 0.69533	valid_0's auc: 0.579603
[42]	training's auc: 0.696122	valid_0's auc: 0.578665
[43]	training's auc: 0.695509	valid_0's auc: 0.57916
[44]	training's auc: 0.69573	valid_0's auc: 0.578701
[45]	training's auc: 0.695897	valid_0's auc: 0.578962
[46]	training's auc: 0.696122	valid_0's auc: 0.578271
[47]	training's auc: 0.696247	valid_0's auc: 0.578577
[48]	training's auc: 0.696371	valid_0's auc: 0.578363
[49]	training's auc: 0.696462	val

In [68]:
print(use_cols)

['city_name', 'county_name', 'idcard_cnt', 'arpu_202004', 'idcard_cnt*arpu_202004', 'city_name_count', 'county_name_count', 'idcard_cnt_count', 'city_name_idcard_cnt_max', 'city_name_idcard_cnt_min', 'city_name_idcard_cnt_median', 'county_name_idcard_cnt_max', 'county_name_idcard_cnt_min', 'county_name_idcard_cnt_median', 'calltype_id', 'voc_dayofweek', 'voc_day_count_max', 'voc_day_count_min', 'voc_day_count_mean', 'voc_day_count_std', 'voc_day_diff_count_max', 'voc_day_diff_count_mean', 'voc_day_diff_count_std', 'voc_day22_count', 'voc_day23_count', 'voc_day1_count', 'voc_day3_count', 'voc_day9_count', 'voc_day6_count', 'voc_day10_count', 'voc_day26_count', 'voc_day27_count', 'voc_day13_count', 'voc_day28_count', 'voc_day30_count', 'voc_day20_count', 'voc_day16_count', 'voc_day19_count', 'voc_day21_count', 'voc_day29_count', 'voc_day17_count', 'voc_day7_count', 'voc_day14_count', 'voc_day15_count', 'voc_day5_count', 'voc_day2_count', 'voc_day12_count', 'voc_day8_count', 'voc_day18_co

In [69]:
print(useless_cols)

['call_dur', 'voc_day', 'voc_hour', 'voc_day_diff_count_min', 'voc_hour_diff_count_min', 'voc_hour_call_dur1_sum', 'voc_hour_call_dur3_sum', 'voc_hour_call_dur4_sum', 'sms_day', 'sms_dayofweek', 'sms_day_diff_count_min', 'sms_hour_count_min', 'sms_hour_diff_count_min']


In [70]:
lgb_train = lgb.Dataset(X_train[use_cols].values, y_train) 

lgb_eval= lgb.Dataset(X_valid[use_cols].values, y_valid, reference=lgb_train)  

print('Start training...')

# train

lgb_val_0 = lgb.train(params,
                      lgb_train,
                      num_boost_round=10000,
                      valid_sets=[lgb_eval, lgb_train],
                      early_stopping_rounds=100,
                      verbose_eval=10)

Start training...
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.993027	valid_0's auc: 0.8204
[20]	training's auc: 0.997845	valid_0's auc: 0.824036
[30]	training's auc: 0.999204	valid_0's auc: 0.83015
[40]	training's auc: 0.99987	valid_0's auc: 0.838281
[50]	training's auc: 0.999997	valid_0's auc: 0.840514
[60]	training's auc: 1	valid_0's auc: 0.841348
[70]	training's auc: 1	valid_0's auc: 0.843691
[80]	training's auc: 1	valid_0's auc: 0.844167
[90]	training's auc: 1	valid_0's auc: 0.847103
[100]	training's auc: 1	valid_0's auc: 0.847415
[110]	training's auc: 1	valid_0's auc: 0.847233
[120]	training's auc: 1	valid_0's auc: 0.847018
[130]	training's auc: 1	valid_0's auc: 0.847565
[140]	training's auc: 1	valid_0's auc: 0.847878
[150]	training's auc: 1	valid_0's auc: 0.848854
[160]	training's auc: 1	valid_0's auc: 0.84735
[170]	training's auc: 1	valid_0's auc: 0.848066
[180]	training's auc: 1	valid_0's auc: 0.848822
[190]	training's auc: 1	valid_0's a

In [71]:
# 验证集结果
X_valid['prob'] = lgb_val_0.predict(X_valid[use_cols])
X_valid['pred'] = np.where(X_valid['prob'] > 0.4735, 1, 0)

f1 = np.round(f1_score(y_valid, X_valid['pred']), 4)
auc = roc_auc_score(y_valid, X_valid['prob'])

print('f1: ', f1)
print('auc: ', auc)

f1:  0.5619
auc:  0.8491796875


In [72]:
# 验证集结果
X_valid['prob'] = lgb_val_0.predict(X_valid[use_cols])
X_valid['pred'] = np.where(X_valid['prob'] > 0.5, 1, 0)

f1 = np.round(f1_score(y_valid, X_valid['pred']), 4)
auc = roc_auc_score(y_valid, X_valid['prob'])

print('f1: ', f1)
print('auc: ', auc)

f1:  0.5673
auc:  0.8491796875


In [73]:
lgb_train_all = lgb.Dataset(df_train[use_cols].values, df_train['label'])   

print('Start training...')

# train

lgb_model = lgb.train(params,
                      lgb_train_all,
                      num_boost_round=lgb_val_0.best_iteration + 20)

Start training...


In [74]:
df_test['label'] = np.where(lgb_model.predict(df_test[use_cols]) > 0.4735, 1, 0)
df_test[['phone_no_m', 'label']].to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), f1), index=False)